**Pandas Notebook 6**

A demonstration of advanced `pandas` syntax to accompany class Lecture.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

#More on `Groupby`





Try to predict the results of the `groupby` operation shown. The answer is below the image.
The top ?? will be "hi", the second ?? will be "tx", and the third ?? will be "sd".

In [ ]:
# Form a data frame using dictionary
# Answer Here
ds = pd.DataFrame(dict(x=[3, 1, 4, 1, 5, 9, 2, 5, 6],
                      y=['ak', 'tx', 'fl', 'hi', 'mi', 'ak', 'ca', 'sd', 'nc']),
                      index=list('ABCABCACB') )
ds

,x,y
A,3,ak
B,1,tx
C,4,fl
A,1,hi
B,5,mi
C,9,ak
A,2,ca
C,5,sd
B,6,nc


In [ ]:
#Use groupby on index and get max of each group
ds.groupby(ds.index).max()

,x,y
A,3,hi
B,6,tx
C,9,sd


### `groupby.size` and `groupby.count()`

In [ ]:
df = pd.DataFrame({'letter':['A', 'A', 'B', 'C', 'C', 'C'],
                   'num':[1, 2, 3, 4, np.NaN, 4],
                   'state':[np.NaN, 'tx', 'fl', 'hi', np.NaN, 'ak']})
df

,letter,num,state
0,A,1.0,NaN
1,A,2.0,tx
2,B,3.0,fl
3,C,4.0,hi
4,C,NaN,NaN
5,C,4.0,ak


`groupby.size()` returns a `Series`, indexed by the `letter`s that we grouped by, with values denoting the number of rows in each group/sub-DataFrame. It does not care about missing (`NaN`) values.

In [ ]:
# Use groupby with size()
df.groupby("letter").size()

,0
letter,
A,2
B,1
C,3


`groupby.count()` returns a `DataFrame`, indexed by the `letter`s that we grouped by. Each column represents the number of non-missing values for that `letter`.

In [ ]:
# Use groupby with count()
df.groupby("letter").count()

,num,state
letter,,
A,2,1
B,1,1
C,2,2


You might recall `value_counts()` function we talked about last week. What's the difference?

In [ ]:
# Use value_count() on DataFrame described above
df.groupby("letter").value_counts()

letter  num  state
A       2.0  tx       1
B       3.0  fl       1
C       4.0  ak       1
             hi       1
Name: count, dtype: int64

Turns out `value_counts()` does something similar to `groupby.size()`, except that it also sorts the resulting `Series` in descending order.

## Filtering by Group

In [ ]:
# Let's read the elections dataset
elections = pd.read_csv("/content/drive/MyDrive/elections.csv")
elections

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
...,...,...,...,...,...,...
177,2016,Jill Stein,Green,1457226,loss,1.073699
178,2020,Joseph Biden,Democratic,81268924,win,51.311515
179,2020,Donald Trump,Republican,74216154,loss,46.858542
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979


Let's keep only the elections years where the maximum vote share `%` is less than 45%.

In [ ]:
# use filter function
election_win_45 = elections.groupby("Year").filter(lambda max:max["%"].max() < 45)
election_win_45

,Year,Candidate,Party,Popular vote,Result,%,First Name
23,1860,Abraham Lincoln,Republican,1855993,win,39.699408,Abraham
24,1860,John Bell,Constitutional Union,590901,loss,12.639283,John
25,1860,John C. Breckinridge,Southern Democratic,848019,loss,18.138998,John
26,1860,Stephen A. Douglas,Northern Democratic,1380202,loss,29.522311,Stephen
66,1912,Eugene V. Debs,Socialist,901551,loss,6.004354,Eugene
67,1912,Eugene W. Chafin,Prohibition,208156,loss,1.386325,Eugene
68,1912,Theodore Roosevelt,Progressive,4122721,loss,27.457433,Theodore
69,1912,William Taft,Republican,3486242,loss,23.218466,William
70,1912,Woodrow Wilson,Democratic,6296284,win,41.933422,Woodrow
115,1968,George Wallace,American Independent,9901118,loss,13.571218,George


### `groupby` Puzzle

Assume that we want to know the best election by each party.

#### Attempt #1

We have to be careful when using aggregation functions. For example, the code below might be misinterpreted to say that Woodrow Wilson successfully ran for election in 2020. Why is this happening?

In [ ]:
# Use agg(max)
elections.groupby("Party").agg(max).head(10)

<ipython-input-40-4cefb6911cca>:2: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  elections.groupby("Party").agg(max).head(10)


,Year,Candidate,Popular vote,Result,%,First Name
Party,,,,,,
American,1976,Thomas J. Anderson,873053,loss,21.554001,Thomas
American Independent,1976,Lester Maddox,9901118,loss,13.571218,Lester
Anti-Masonic,1832,William Wirt,100715,loss,7.821583,William
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838,Benjamin
Citizens,1980,Barry Commoner,233052,loss,0.270182,Barry
Communist,1932,William Z. Foster,103307,loss,0.261069,William
Constitution,2016,Michael Peroutka,203091,loss,0.152398,Michael
Constitutional Union,1860,John Bell,590901,loss,12.639283,John
Democratic,2020,Woodrow Wilson,81268924,win,61.344703,Woodrow


#### Attempt #2

Next, we'll write code that properly returns _the best result by each party_. That is, each row should show the Year, Candidate, Popular Vote, Result, and % for the election in which that party saw its best results (rather than mixing them as in the example above). Here's what the first rows of the correct output should look like:


In [ ]:
# Use sort and then groupby as describe in the lec
elections_sorted = elections.sort_values(by="%", ascending=False)
elections_sorted

,Year,Candidate,Party,Popular vote,Result,%,First Name
114,1964,Lyndon Johnson,Democratic,43127041,win,61.344703,Lyndon
91,1936,Franklin Roosevelt,Democratic,27752648,win,60.978107,Franklin
120,1972,Richard Nixon,Republican,47168710,win,60.907806,Richard
79,1920,Warren Harding,Republican,16144093,win,60.574501,Warren
133,1984,Ronald Reagan,Republican,54455472,win,59.023326,Ronald
...,...,...,...,...,...,...,...
165,2008,Cynthia McKinney,Green,161797,loss,0.123442,Cynthia
148,1996,John Hagelin,Natural Law,113670,loss,0.118219,John
160,2004,Michael Peroutka,Constitution,143630,loss,0.117542,Michael
141,1992,Bo Gritz,Populist,106152,loss,0.101918,Bo


In [ ]:
# grab first entry
elections_sorted.groupby("Party").first()

,Year,Candidate,Popular vote,Result,%,First Name
Party,,,,,,
American,1856,Millard Fillmore,873053,loss,21.554001,Millard
American Independent,1968,George Wallace,9901118,loss,13.571218,George
Anti-Masonic,1832,William Wirt,100715,loss,7.821583,William
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838,Benjamin
Citizens,1980,Barry Commoner,233052,loss,0.270182,Barry
Communist,1932,William Z. Foster,103307,loss,0.261069,William
Constitution,2008,Chuck Baldwin,199750,loss,0.152398,Chuck
Constitutional Union,1860,John Bell,590901,loss,12.639283,John
Democratic,1964,Lyndon Johnson,43127041,win,61.344703,Lyndon


#### Alternative Solutions

You'll soon discover that with Pandas rich tool set, there's typically more than one way to get to the same answer. Each approach has different tradeoffs in terms of readability, performance, memory consumption, complexity, and more. It will take some experience for you to develop a sense of which approach is better for each problem, but you should, in general, try to think if you can at least envision a different solution to a given problem, especially if you find your current solution to be particularly convoluted or hard to read.

Here are a couple of other ways of obtaining the same result (in each case, we only show the top part with `head()`). The first approach uses `groupby` but finds the location of the maximum value via the `idxmax()` method (look up its documentation!).  We then index and sort by `Party` to match the requested formatting:

In [ ]:
# Use idxmax function
using_idmax = elections_sorted.loc[elections_sorted.groupby("Party")["%"].idxmax()]
using_idmax

,Year,Candidate,Party,Popular vote,Result,%,First Name
22,1856,Millard Fillmore,American,873053,loss,21.554001,Millard
115,1968,George Wallace,American Independent,9901118,loss,13.571218,George
6,1832,William Wirt,Anti-Masonic,100715,loss,7.821583,William
38,1884,Benjamin Butler,Anti-Monopoly,134294,loss,1.335838,Benjamin
127,1980,Barry Commoner,Citizens,233052,loss,0.270182,Barry
89,1932,William Z. Foster,Communist,103307,loss,0.261069,William
164,2008,Chuck Baldwin,Constitution,199750,loss,0.152398,Chuck
24,1860,John Bell,Constitutional Union,590901,loss,12.639283,John
114,1964,Lyndon Johnson,Democratic,43127041,win,61.344703,Lyndon
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,Andrew


In [ ]:
  # This is the computational part, Extract DataFrame based on above mentioned condition

# Apply sort_index to match the formatting above
using_idxmax_sorted = using_idmax.sort_index()
using_idxmax_sorted

,Year,Candidate,Party,Popular vote,Result,%,First Name
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,Andrew
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,John
6,1832,William Wirt,Anti-Masonic,100715,loss,7.821583,William
11,1840,William Henry Harrison,Whig,1275583,win,53.051213,William
15,1848,Martin Van Buren,Free Soil,291501,loss,10.138474,Martin
22,1856,Millard Fillmore,American,873053,loss,21.554001,Millard
24,1860,John Bell,Constitutional Union,590901,loss,12.639283,John
25,1860,John C. Breckinridge,Southern Democratic,848019,loss,18.138998,John
26,1860,Stephen A. Douglas,Northern Democratic,1380202,loss,29.522311,Stephen
27,1864,Abraham Lincoln,National Union,2211317,win,54.951512,Abraham


Another approach is listed below. And this one doesn't even use `groupby`!

This approach instead uses the `drop_duplicates` method to keep only the last occurrence of of each party after having sorted by "%", which is the best performance.

In [ ]:

# Formatting, Again, the 2nd line is purely formatting:
best_per_party2 = elections.sort_values("%").drop_duplicates(["Party"], keep="last")
best_per_party2

,Year,Candidate,Party,Popular vote,Result,%,First Name
148,1996,John Hagelin,Natural Law,113670,loss,0.118219,John
164,2008,Chuck Baldwin,Constitution,199750,loss,0.152398,Chuck
110,1956,T. Coleman Andrews,States' Rights,107929,loss,0.174883,T.
147,1996,Howard Phillips,Taxpayers,184656,loss,0.192045,Howard
136,1988,Lenora Fulani,New Alliance,217221,loss,0.237804,Lenora
89,1932,William Z. Foster,Communist,103307,loss,0.261069,William
127,1980,Barry Commoner,Citizens,233052,loss,0.270182,Barry
50,1896,John M. Palmer,National Democratic,134645,loss,0.969566,John
78,1920,Parley P. Christensen,Farmer–Labor,265398,loss,0.995804,Parley
42,1888,Alson Streeter,Union Labor,146602,loss,1.288861,Alson


*Challenge:* See if you can find yet another approach that still gives the same answer.

### `DataFrameGroupBy` Objects

The result of `groupby` is not a `DataFrame` or a list of `DataFrame`s. It is instead a special type called a `DataFrameGroupBy`.

In [ ]:
grouped_by_party = elections.groupby("Party")
type(grouped_by_party)
elections

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
...,...,...,...,...,...,...
177,2016,Jill Stein,Green,1457226,loss,1.073699
178,2020,Joseph Biden,Democratic,81268924,win,51.311515
179,2020,Donald Trump,Republican,74216154,loss,46.858542
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979


`GroupBy` objects are structured like dictionaries. In fact, we can actually see the dictionaries with the following code:

In [ ]:
# visualize all groups using .groups
grouped_by_party.groups

{'American': [22, 126], 'American Independent': [115, 119, 124], 'Anti-Masonic': [6], 'Anti-Monopoly': [38], 'Citizens': [127], 'Communist': [89], 'Constitution': [160, 164, 172], 'Constitutional Union': [24], 'Democratic': [2, 4, 8, 10, 13, 14, 17, 20, 28, 29, 34, 37, 39, 45, 47, 52, 55, 57, 64, 70, 74, 77, 81, 83, 86, 91, 94, 97, 100, 105, 108, 111, 114, 116, 118, 123, 129, 134, 137, 140, 144, 151, 158, 162, 168, 176, 178], 'Democratic-Republican': [0, 1], 'Dixiecrat': [103], 'Farmer–Labor': [78], 'Free Soil': [15, 18], 'Green': [149, 155, 156, 165, 170, 177, 181], 'Greenback': [35], 'Independent': [121, 130, 143, 161, 167, 174], 'Liberal Republican': [31], 'Libertarian': [125, 128, 132, 138, 139, 146, 153, 159, 163, 169, 175, 180], 'National Democratic': [50], 'National Republican': [3, 5], 'National Union': [27], 'Natural Law': [148], 'New Alliance': [136], 'Northern Democratic': [26], 'Populist': [48, 61, 141], 'Progressive': [68, 82, 101, 107], 'Prohibition': [41, 44, 49, 51, 54,

The `key`s of the dictionary are the groups (in this case, `Party`), and the `value`s are the **indices** of rows belonging to that group. We can access a particular sub-`DataFrame` using `get_group`:

In [ ]:
# use get_group to extract 'Socialists' party results
grouped_by_party.get_group('Socialist')

,Year,Candidate,Party,Popular vote,Result,%
58,1904,Eugene V. Debs,Socialist,402810,loss,2.985897
62,1908,Eugene V. Debs,Socialist,420852,loss,2.850866
66,1912,Eugene V. Debs,Socialist,901551,loss,6.004354
71,1916,Allan L. Benson,Socialist,590524,loss,3.194193
76,1920,Eugene V. Debs,Socialist,913693,loss,3.428282
85,1928,Norman Thomas,Socialist,267478,loss,0.728623
88,1932,Norman Thomas,Socialist,884885,loss,2.236211
92,1936,Norman Thomas,Socialist,187910,loss,0.412876
95,1940,Norman Thomas,Socialist,116599,loss,0.234237
102,1948,Norman Thomas,Socialist,139569,loss,0.286312


## Pivot Tables

### `Groupby` with multiple columns

We want to build a table showing the total number of babies born of each sex in each year. One way is to `groupby` using both columns of interest:

In [ ]:
# Find total count of baby names for both female and Male for each year
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "D:\Workbook assignments\babynamesbystate.zip"
if not os.path.exists(local_filename): # If the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.head()
babynames.groupby(['Year','Sex'])[['Count']].agg(sum).head(6)

<ipython-input-25-40d674b27b5c>:20: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  babynames.groupby(['Year','Sex'])[['Count']].agg(sum).head(6)


Count
Year Sex       
1910 F     5950
     M     3213
1911 F     6602
     M     3381
1912 F     9804
     M     8142

### `pivot_table`

In [ ]:
# Find total count of baby names for both female and Male for each year using Pivot table
babynames_pivot = babynames.pivot_table(
    index= 'Year',
    columns= 'Sex',
    values= ['Count'],
    aggfunc= np.sum
)
babynames_pivot.head(6)

<ipython-input-19-a383648bed05>:2: FutureWarning: The provided callable <function sum at 0x794feff83250> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  babynames_pivot = babynames.pivot_table(


Count       
Sex       F      M
Year              
1910   5950   3213
1911   6602   3381
1912   9804   8142
1913  11860  10234
1914  13815  13111
1915  18643  17192

### `pivot_table` with Multiple values

In [ ]:
# Form a pivot table as describe in Lecture Slides
babynames_pivot = babynames.pivot_table(
    index= 'Year',
    columns= 'Sex',
    values= ['Count','Name'],
    aggfunc= np.max
)
babynames_pivot.head(6)

<ipython-input-20-8ff37a00f43c>:2: FutureWarning: The provided callable <function max at 0x794feff83880> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  babynames_pivot = babynames.pivot_table(


Count          Name         
Sex      F     M       F        M
Year                             
1910   295   237  Yvonne  William
1911   390   214   Zelma   Willis
1912   534   501  Yvonne  Woodrow
1913   584   614   Zelma   Yoshio
1914   773   769   Zelma   Yoshio
1915   998  1033    Zita    Yukio

## Join Tables

What if we want to know the popularity of presidential candidates' first names in California in 2022? What can we do?

In [ ]:
elections.head(10)

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
5,1832,Henry Clay,National Republican,484205,loss,37.603628
6,1832,William Wirt,Anti-Masonic,100715,loss,7.821583
7,1836,Hugh Lawson White,Whig,146109,loss,10.005985
8,1836,Martin Van Buren,Democratic,763291,win,52.272472
9,1836,William Henry Harrison,Whig,550816,loss,37.721543


In [ ]:
# Collect baby names for 2022
babynames_2022 = babynames[babynames['Year'] == 2022]
babynames_2022

,State,Sex,Year,Name,Count
235861,CA,F,2022,Olivia,2184
235862,CA,F,2022,Emma,2086
235863,CA,F,2022,Camila,2048
235864,CA,F,2022,Mia,1890
235865,CA,F,2022,Sophia,1770
...,...,...,...,...,...
411107,CA,M,2022,Zia,5
411108,CA,M,2022,Zora,5
411109,CA,M,2022,Zuri,5
411110,CA,M,2022,Zuriel,5


In [ ]:
# Use split the candidate names in elections dataframe
elections['First Name'] = elections['Candidate'].str.split().str[0]
elections

,Year,Candidate,Party,Popular vote,Result,%,First Name
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,Andrew
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878,John
2,1828,Andrew Jackson,Democratic,642806,win,56.203927,Andrew
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,John
4,1832,Andrew Jackson,Democratic,702735,win,54.574789,Andrew
...,...,...,...,...,...,...,...
177,2016,Jill Stein,Green,1457226,loss,1.073699,Jill
178,2020,Joseph Biden,Democratic,81268924,win,51.311515,Joseph
179,2020,Donald Trump,Republican,74216154,loss,46.858542,Donald
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979,Jo


`join` in pandas

In [ ]:
#Merge both elections and babynames and report your analysis
merged = pd.merge(left=elections,right=babynames_2022,left_on="First Name",right_on="Name")
merged

#Analysis
#1. Many historical politicians name are still common in year 2022
#2. Both the losing and winning candidate names are common still now
#3. Many names lose there popularity which shows change in trends

,Year_x,Candidate,Party,Popular vote,Result,%,First Name,State,Sex,Year_y,Name,Count
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,Andrew,CA,M,2022,Andrew,743
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878,John,CA,M,2022,John,494
2,1828,Andrew Jackson,Democratic,642806,win,56.203927,Andrew,CA,M,2022,Andrew,743
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,John,CA,M,2022,John,494
4,1832,Andrew Jackson,Democratic,702735,win,54.574789,Andrew,CA,M,2022,Andrew,743
...,...,...,...,...,...,...,...,...,...,...,...,...
149,2016,Gary Johnson,Libertarian,4489235,loss,3.307714,Gary,CA,M,2022,Gary,20
150,2016,Hillary Clinton,Democratic,65853514,loss,48.521539,Hillary,CA,F,2022,Hillary,10
151,2020,Joseph Biden,Democratic,81268924,win,51.311515,Joseph,CA,M,2022,Joseph,792
152,2020,Donald Trump,Republican,74216154,loss,46.858542,Donald,CA,M,2022,Donald,33


In [ ]:
# Sort using Count
merged.sort_values(by="Count")

,Year_x,Candidate,Party,Popular vote,Result,%,First Name,State,Sex,Year_y,Name,Count
145,2016,Darrell Castle,Constitution,203091,loss,0.149640,Darrell,CA,M,2022,Darrell,5
78,1928,Herbert Hoover,Republican,21427123,win,58.368524,Herbert,CA,M,2022,Herbert,5
81,1932,Herbert Hoover,Republican,15761254,loss,39.830594,Herbert,CA,M,2022,Herbert,5
69,1916,Woodrow Wilson,Democratic,9126868,win,49.367987,Woodrow,CA,M,2022,Woodrow,6
99,1964,Lyndon Johnson,Democratic,43127041,win,61.344703,Lyndon,CA,M,2022,Lyndon,6
...,...,...,...,...,...,...,...,...,...,...,...,...
38,1884,James G. Blaine,Republican,4856905,loss,48.312208,James,CA,M,2022,James,1094
14,1844,James Polk,Democratic,1339570,win,50.749477,James,CA,M,2022,James,1094
36,1884,Benjamin Butler,Anti-Monopoly,134294,loss,1.335838,Benjamin,CA,M,2022,Benjamin,1528
42,1892,Benjamin Harrison,Republican,5176108,loss,42.984101,Benjamin,CA,M,2022,Benjamin,1528
